# 🚀 Quickstart: How to Use This Notebook

1. Put your datasets (CSV or Excel) in a folder.  
   Example: `C:/Users/You/Documents/Datasets`

2. Update the `DATA_FOLDER` path in the CONFIGURATION section below
   to point to that folder.

3. Run the notebook cells step by step.

4. What you’ll get in the `output` folder:
   - ✅ Cleaned dataset (Excel file)
   - 📊 Profiling report (HTML with stats & visuals)
   - 📈 Charts (PNG images)

💡 Tip: If `DATA_FOLDER` is left empty/None, the notebook will prompt you
to enter a folder path interactively.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from ydata_profiling import ProfileReport
import warnings
from io import StringIO
from contextlib import redirect_stdout

# Suppress warnings from matplotlib and pandas for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# -----------------------------
# CONFIGURATION
# -----------------------------

# =============================
# ⚠️ How to use this section:
# - Change `DATA_FOLDER` to where your datasets actually live.
# - Change `OUTPUT_FOLDER` if you want results saved elsewhere.
# - Set SHOW_PREVIEW = False if you don’t want sample rows printed.
# - Usually you don’t need to touch EXCLUDE_PATTERNS unless you 
#   notice weird columns in your plots.
# =============================

# Path to the folder where your datasets are stored
DATA_FOLDER = r"C:\Users\Admin\Documents\Automation\Datasets"  # <-- IMPORTANT: SET YOUR DATA FOLDER PATH

# Path where all outputs (cleaned files, plots, reports) will be saved
OUTPUT_FOLDER = "output"

# Whether to show quick previews (first 5 rows) when loading data
SHOW_PREVIEW = True 

# Columns to EXCLUDE automatically from visualization (too generic / irrelevant)
EXCLUDE_PATTERNS = ['id', 'desc', 'number', 'phone', 'contact', 'name']

# =============================================================================
# MODULAR FUNCTIONS
# =============================================================================

def list_and_select_files(data_folder: str) -> list:
    """
    Scans the data folder for datasets, displays them, and prompts the user
    to select one or more files for processing.

    Args:
        data_folder (str): The path to the folder containing datasets.

    Returns:
        list: A list of full file paths for the selected datasets.
    """
    try:
        files = [f for f in os.listdir(data_folder) if f.lower().endswith((".xlsx", ".xls", ".csv"))]
    except FileNotFoundError:
        print(f"❌ ERROR: The specified data folder does not exist: {data_folder}")
        return []

    if not files:
        print(f"❌ No datasets found in: {data_folder}")
        return []

    print("\nAvailable datasets:")
    for i, f in enumerate(files, 1):
        print(f"  {i}. {f}")

    choice_str = input(
        "\nEnter number(s) of dataset(s) to process, comma-separated (e.g., 1,3 or 2) [default: last]: "
    ).strip()

    selected_files = []
    if not choice_str:
        selected_files.append(os.path.join(data_folder, files[-1])) # Default to last file
    else:
        try:
            choices = [int(c.strip()) - 1 for c in choice_str.split(",")]
            selected_files = [os.path.join(data_folder, files[i]) for i in choices]
        except (ValueError, IndexError):
            print("⚠️ Invalid input. Defaulting to the last dataset.")
            selected_files.append(os.path.join(data_folder, files[-1]))

    return selected_files


def load_data(file_path: str) -> pd.DataFrame | None:
    """
    Loads a dataset from a given file path (CSV or Excel).

    Args:
        file_path (str): The full path to the data file.

    Returns:
        pd.DataFrame | None: A pandas DataFrame if successful, otherwise None.
    """
    print(f"\n⏳ Loading data from: {os.path.basename(file_path)}")
    try:
        if file_path.lower().endswith(".csv"):
            df = pd.read_csv(file_path, encoding="latin1")
        else:
            df = pd.read_excel(file_path)
        print(f"✅ Loaded successfully with shape {df.shape}")
        return df
    except Exception as e:
        print(f"❌ Error loading {os.path.basename(file_path)}: {e}")
        return None


def get_cleaning_choices(df_columns: list) -> dict:
    """
    Asks the user for their preferred data cleaning methods and optional column selection for visualizations.

    Args:
        df_columns (list): List of DataFrame columns to validate visualization choices.

    Returns:
        dict: A dictionary containing the user's cleaning and visualization choices.
    """
    print("\n--- Data Cleaning Options ---")
    
    # Choice for handling missing values
    while True:
        mv_choice = input("How to handle missing values?\n  [1] Drop rows with missing values\n  [2] Fill with mean (numeric) / mode (text)\n  [3] Do nothing\n  Enter your choice (1/2/3): ").strip()
        if mv_choice in ['1', '2', '3']:
            break
        print("⚠️ Invalid choice. Please enter 1, 2, or 3.")

    # Choice for handling duplicates
    while True:
        dd_choice = input("Drop duplicate rows? (y/n): ").strip().lower()
        if dd_choice in ['y', 'n']:
            break
        print("⚠️ Invalid choice. Please enter y or n.")

    # Choice for manual column selection for visualizations
    print("\n--- Visualization Options ---")
    print(f"Available columns: {df_columns}")
    bar_cols = []
    pie_cols = []
    while True:
        manual_choice = input("Manually select columns for visualizations? (y/n): ").strip().lower()
        if manual_choice in ['y', 'n']:
            break
        print("⚠️ Invalid choice. Please enter y or n.")

    if manual_choice == 'y':
        # Bar chart columns
        bar_input = input("Enter column names for bar charts (comma-separated, e.g., Vehicle_Type,City) or 'none': ").strip()
        if bar_input.lower() != 'none':
            bar_cols = [col.strip() for col in bar_input.split(",")]
            bar_cols = [col for col in bar_cols if col in df_columns]
            if not bar_cols:
                print("⚠️ No valid columns selected for bar charts. Using auto-selection.")
            else:
                print(f"Selected bar chart columns: {bar_cols}")

        # Pie chart columns
        pie_input = input("Enter column names for pie charts (comma-separated, e.g., Booking_Status,Payment_Method) or 'none': ").strip()
        if pie_input.lower() != 'none':
            pie_cols = [col.strip() for col in pie_input.split(",")]
            pie_cols = [col for col in pie_cols if col in df_columns]
            if not pie_cols:
                print("⚠️ No valid columns selected for pie charts. Using auto-selection.")
            else:
                print(f"Selected pie chart columns: {pie_cols}")

    choices_map = {'1': 'drop', '2': 'fill', '3': 'none'}
    return {
        'missing_values': choices_map[mv_choice],
        'drop_duplicates': True if dd_choice == 'y' else False,
        'bar_columns': bar_cols,
        'pie_columns': pie_cols
    }


def clean_data(df: pd.DataFrame, choices: dict) -> tuple[pd.DataFrame, dict]:
    """
    Cleans the DataFrame based on the user's choices and returns a mapping of original to cleaned column names.

    Args:
        df (pd.DataFrame): The input DataFrame.
        choices (dict): A dictionary of cleaning options.

    Returns:
        tuple: (Cleaned DataFrame, dictionary mapping original to cleaned column names).
    """
    print("⏳ Applying cleaning rules...")
    df_cleaned = df.copy()
    
    # Store original column names
    original_columns = df_cleaned.columns.tolist()
    
    # 1. Handle Duplicates
    if choices['drop_duplicates']:
        initial_rows = len(df_cleaned)
        df_cleaned.drop_duplicates(inplace=True)
        print(f"  - Dropped {initial_rows - len(df_cleaned)} duplicate rows.")

    # 2. Handle Missing Values
    if choices['missing_values'] == 'drop':
        initial_rows = len(df_cleaned)
        df_cleaned.dropna(inplace=True)
        print(f"  - Dropped {initial_rows - len(df_cleaned)} rows with missing values.")
    elif choices['missing_values'] == 'fill':
        for col in df_cleaned.columns:
            if df_cleaned[col].isnull().any():
                if pd.api.types.is_numeric_dtype(df_cleaned[col]):
                    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mean())
                else:
                    df_cleaned[col] = df_cleaned[col].fillna(
                        df_cleaned[col].mode().iloc[0] if not df_cleaned[col].mode().empty else "Unknown"
                    )
        print("  - Filled missing values with mean/mode.")

    # 3. Standardize Column Names
    cleaned_columns = [c.strip().replace(" ", "_") for c in df_cleaned.columns]
    column_mapping = dict(zip(original_columns, cleaned_columns))
    df_cleaned.columns = cleaned_columns
    print("  - Standardized column names.")

    # 4. Attempt to convert data types
    def try_to_numeric(series):
        try:
            return pd.to_numeric(series)
        except (ValueError, TypeError):
            return series
    df_cleaned = df_cleaned.apply(try_to_numeric)
    print("  - Converted applicable columns to numeric types.")
    
    print(f"✅ Cleaning complete. New shape: {df_cleaned.shape}")
    return df_cleaned, column_mapping


def generate_visualizations(df: pd.DataFrame, base_filename: str, output_folder: str, bar_columns: list, pie_columns: list):
    """
    Generates 2-4 visualizations (1-2 bar charts for top categories, 1-2 pie charts for distributions) 
    using user-selected columns or automatic selection.

    Args:
        df (pd.DataFrame): The DataFrame to visualize (preferably cleaned).
        base_filename (str): The base name for output files.
        output_folder (str): The main output directory.
        bar_columns (list): User-selected columns for bar charts, if any.
        pie_columns (list): User-selected columns for pie charts, if any.
    """
    print("📊 Generating visualizations...")
    plot_folder = os.path.join(output_folder, f"{base_filename}_plots")
    os.makedirs(plot_folder, exist_ok=True)

    # Get categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    # Filter out excluded columns based on patterns for auto-selection
    filtered_cols = [
        col for col in categorical_cols
        if not any(pat.lower() in col.lower() for pat in EXCLUDE_PATTERNS)
    ]

    # Initialize bar and pie columns
    selected_bar_cols = bar_columns if bar_columns else []
    selected_pie_cols = pie_columns if pie_columns else []

    # Automatic selection if no user input
    if not selected_bar_cols and not selected_pie_cols:
        # Pie candidates: low cardinality (2-10 unique values) for distributions
        pie_candidates = sorted([col for col in filtered_cols if 2 <= df[col].nunique() <= 10])
        # Bar candidates: moderate cardinality (5-50 unique values) for top categories
        bar_candidates = sorted([col for col in filtered_cols if 5 <= df[col].nunique() <= 100])

        # Select up to 2 for pie
        selected_pie_cols = pie_candidates[:2]
        # Select up to 2 for bar, preferring those not in pie
        selected_bar_cols = [col for col in bar_candidates if col not in selected_pie_cols][:2]
        # Fallback: If no bar candidates, use remaining filtered cols with >1 unique
        if len(selected_bar_cols) < 1:
            remaining = [col for col in filtered_cols if col not in selected_pie_cols and df[col].nunique() > 1]
            selected_bar_cols = sorted(remaining)[:2]

    visual_count = 0

    # Generate 1-2 Bar Charts (top 10 values)
    for col in selected_bar_cols[:2]:  # Limit to 2
        if col in df.columns:
            plt.figure(figsize=(10, 6))
            top_10 = df[col].value_counts().nlargest(10)
            sns.barplot(x=top_10.values, y=top_10.index).set_title(f'Top 10 {col}')
            plt.xlabel('Count')
            plt.ylabel(col)
            plt.tight_layout()
            plt.savefig(os.path.join(plot_folder, f'bar_{col}.png'))
            plt.close()
            visual_count += 1
            print(f"  - Generated bar chart for {col}")
        else:
            print(f"  ⚠️ Skipping bar chart for {col}: column not found")

    # Generate 1-2 Pie Charts
    for col in selected_pie_cols[:2]:  # Limit to 2
        if col in df.columns and 2 <= df[col].nunique() <= 10:
            plt.figure(figsize=(8, 8))
            value_counts = df[col].value_counts()
            plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
            plt.title(f'Distribution of {col}')
            plt.savefig(os.path.join(plot_folder, f'pie_{col}.png'))
            plt.close()
            visual_count += 1
            print(f"  - Generated pie chart for {col}")
        else:
            print(f"  ⚠️ Skipping pie chart for {col}: {'column not found' if col not in df.columns else 'invalid number of unique values'}")

    # Fallback: Auto-generate if fewer than 2 visuals
    if visual_count < 2 and not (selected_bar_cols or selected_pie_cols):
        # Pie candidates
        pie_candidates = sorted([col for col in filtered_cols if 2 <= df[col].nunique() <= 10])
        for col in pie_candidates[:2]:
            if visual_count >= 4:
                break
            plt.figure(figsize=(8, 8))
            value_counts = df[col].value_counts()
            plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
            plt.title(f'Distribution of {col}')
            plt.savefig(os.path.join(plot_folder, f'pie_{col}.png'))
            plt.close()
            visual_count += 1
            print(f"  - Generated pie chart for {col} (auto-selected)")

        # Bar candidates
        bar_candidates = sorted([col for col in filtered_cols if 5 <= df[col].nunique() <= 100 and col not in pie_candidates])
        for col in bar_candidates[:2]:
            if visual_count >= 4:
                break
            plt.figure(figsize=(10, 6))
            top_10 = df[col].value_counts().nlargest(10)
            sns.barplot(x=top_10.values, y=top_10.index).set_title(f'Top 10 {col}')
            plt.xlabel('Count')
            plt.ylabel(col)
            plt.tight_layout()
            plt.savefig(os.path.join(plot_folder, f'bar_{col}.png'))
            plt.close()
            visual_count += 1
            print(f"  - Generated bar chart for {col} (auto-selected)")

    if visual_count == 0:
        print("  ⚠️ No visualizations generated: no suitable columns selected or found.")
    
    print(f"✅ Saved {visual_count} plots -> {plot_folder}")


def generate_ydata_report(df: pd.DataFrame, base_filename: str, output_folder: str):
    """
    Generates a ydata-profiling HTML report.

    Args:
        df (pd.DataFrame): The DataFrame to profile (original data is best).
        base_filename (str): The base name for the output file.
        output_folder (str): The directory to save the report in.
    """
    print("📑 Generating interactive HTML report...")
    try:
        with redirect_stdout(StringIO()):  # Suppress stdout to avoid ydata-sdk advertisement
            profile = ProfileReport(df, title=f"Profiling Report for {base_filename}", explorative=True, progress_bar=False)
            report_path = os.path.join(output_folder, f"{base_filename}_report_{datetime.now().strftime('%Y%m%d')}.html")
            profile.to_file(report_path, silent=True)
        print(f"✅ Saved HTML report -> {report_path}")
    except Exception as e:
        print(f"❌ Could not generate ydata-profiling report: {e}")


def save_outputs(df_cleaned: pd.DataFrame, base_filename: str, output_folder: str):
    """
    Saves the cleaned DataFrame to an Excel file.

    Args:
        df_cleaned (pd.DataFrame): The cleaned data.
        base_filename (str): The base name for the output file.
        output_folder (str): The directory to save the file in.
    """
    print("💾 Saving cleaned data...")
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    cleaned_path = os.path.join(output_folder, f"{base_filename}_cleaned_{timestamp}.xlsx")
    df_cleaned.to_excel(cleaned_path, index=False)
    print(f"✅ Saved cleaned dataset -> {cleaned_path}")


# =============================================================================
# MAIN EXECUTION SCRIPT
# =============================================================================

def main():
    """Main function to orchestrate the data processing workflow."""
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    selected_files = list_and_select_files(DATA_FOLDER)
    if not selected_files:
        print("\nNo files selected. Exiting.")
        return

    for file_path in selected_files:
        base_filename = os.path.splitext(os.path.basename(file_path))[0]
        print(f"\n{'='*20} PROCESSING: {base_filename} {'='*20}")
        
        # Load
        raw_df = load_data(file_path)
        if raw_df is None:
            continue # Skip to next file if loading failed

        # Clean and get column mapping
        cleaned_df, column_mapping = clean_data(raw_df, {'missing_values': 'none', 'drop_duplicates': False})

        # Get cleaning and visualization choices with cleaned column names
        cleaning_choices = get_cleaning_choices(cleaned_df.columns)

        # Apply user-selected cleaning
        cleaned_df, _ = clean_data(cleaned_df, cleaning_choices)
        
        # Generate Outputs
        save_outputs(cleaned_df, base_filename, OUTPUT_FOLDER)
        generate_visualizations(
            cleaned_df,
            base_filename,
            OUTPUT_FOLDER,
            cleaning_choices['bar_columns'],
            cleaning_choices['pie_columns']
        )
        generate_ydata_report(raw_df, base_filename, OUTPUT_FOLDER) # Profile raw data
        
    print("\n🎉 Automation complete for all selected files!")


if __name__ == "__main__":
    main()